In [1]:
import utils
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

 # Exercise 2

The solution is given using the matrix formalism.

## II.1 Polynomial fitting

1. Implement $f(x, w) = \Phi^\top w$. First implement $\Phi$ as:

In [1]:
def Phi_f(x, D):
    """Computes the feature matrix [ x^0 x^1 ... x^D ]^T
    x: (N,) samples
    D: exponent to use
    Output:
        (D+1)xN matrix
        """
    return x.reshape(1, -1) ** np.arange(D+1).reshape(-1, 1)

In [2]:
# then $f$:
def f(x, w):
    """Compute the features
    x: (N,) vector of samples
    w: (D+1,) vector of weights
    Output:
        N vector
        """
    D = w.shape[0] -1
    Phi = Phi_f(x, D)  # the exponents of x, size D+1 x N
    y = Phi.T @ w  # of size N,
    return y

2. Implement $E_\lambda(w)$

In [ ]:
def  E(w, lmbda, x, t):
    """Energy function E_\lambda, vectorized
    w: (D,)
    x: (N,D)
    t: (N,)
    """
    y = f(x, w)  # (N,)
    return 1/2*np.linalg.norm(y - t)**2 + lmbda/2 * np.linalg.norm(w)^2

In [ ]:
# and $\nabla E_\lambda(w)$
def grad_E(w, lmbda, x, t):
    """Gradient of E
    w: (D+1,)
    x: (N,)
    t: (N,)
    """
    D = w.shape[0]-1
    Phi = Phi_f(x, D)
    I = np.eye(D)
    return Phi @(Phi.T @ w - t) + lmbda*w

3.a. Gradient descent steps to minimize $E$.

In [ ]:
def find_w_GD(w0,  lmbda, x, t, eta=0.1, tol=1e-3, MAX_STEP=10_000):
    """Find the optimal w with gradient descent
    w0: (D+1,) starting point
    x: (N,) samples
    t: (N,) targets
    tol: tolerance as stoppping criterion
    Output:
        w* optimal
    """
    k = 0
    converged = False
    w = w0
    while not converged:
        grad = grad_E(w, lmbda, x, t)
        w = w - eta * grad
        converged = np.linalg.norm(grad) < tol or k > MAX_STEP
        k += 1
    return w, k

3.b Find $w^*$ by solving the linear system, $w^* = (\Phi \Phi^\top + \lambda
I)^{-1} \Phi t$

In [ ]:
def find_w_sys(lmbda, x, t, D):
    """Find the optimal w by solving the system of linear equations
    x: (N,) samples
    t: (N,) targets
    D: degree of the polynomial
    """
    I = np.eye(D+1)
    Phi = Phi_f(x, D)
    Inv = sp.linalg.pinv(Phi @ Phi.T + lmbda* I)  # pseudo inverse, in the case it is not invertible
    return Inv @ Phi @ t

Plot the different results

In [ ]:
# Fixed N, varying lambda, D
Ds = [1, 3, 9, 15]  # will serve as row index
lambdas = [0, 0.01, 0.1, 1]  # column index
N = 30  # number of samples
eta = 0.1  # learning rate

In [ ]:
np.random.seed(1)

In [ ]:
Nrow = len(Ds)
Ncol = len(lambdas)
x, t = utils.gen_sin_data(N=N)  # training samples, N points
fig, axes = plt.subplots(Nrow, Ncol, figsize=(Ncol*4, Nrow*4))
for iD, D in enumerate(Ds):
    for il, lmbda in enumerate(lambdas):


        ax = axes[iD,il]
        w0 = np.zeros(D+1)  # y=0, starting point for GD
        w_GD, nsteps = find_w_GD(w0, lmbda, x, t, eta=eta)  # solution from GD

        w_sys = find_w_sys(lmbda, x, t, D)  # solution from solving linear system

        xtest = np.linspace(min(x),max(x))
        ytest_GD = f(xtest, w_GD)
        ytest_sys = f(xtest, w_sys)

        ax.scatter(x, t, label='samples')
        ax.plot(xtest, ytest_GD, color='orange', label=f"GD ({nsteps} iter)")
        ax.plot(xtest, ytest_sys, color='red', label="Sol. system")
        ax.legend()
        ax.set_title(f"$\lambda={lmbda}, D={D}$")

In [ ]:
fig.suptitle(f"N = {N}")
plt.tight_layout()
plt.show()

**Questions:** 1.What happens when D> N?
2.What can go wrong with the linear system solution? With the gradient
solution?
3.How to correct these issues?